# 损失函数

[![](https://gitee.com/mindspore/docs/raw/r1.3/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.3/tutorials/source_zh_cn/intermediate/custom/loss.ipynb)

损失函数用于衡量预测值与真实值差异的程度。深度学习中，模型训练就是通过不停地迭代来缩小损失函数值的过程。因此在模型训练过程中损失函数的选择非常重要，定义一个好的损失函数，可以有效提高模型的性能。

MindSpore提供了许多通用损失函数供用户选择，但这些通用损失函数并不适用于所有场景，有时需要用户自定义所需的损失函数。因此，本文介绍损失函数的自定义构建方法。

## 自定义损失函数

Cell是MindSpore的基本网络单元，可以用于构建网络，损失函数也需要通过Cell来定义。使用Cell定义损失函数的方法与定义一个普通的网络相同，差别在于，其执行逻辑用于计算前向网络输出与真实值之间的误差。

以MindSpore提供的损失函数`L1Loss`为例，损失函数的定义方法如下：

```python
import mindspore.nn as nn
import mindspore.ops as ops

class L1Loss(nn.Cell):
    def __init__(self):
        super(L1Loss, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()

    def construct(self, predict, target):
        x = self.abs(predict - target)
        return self.reduce_mean(x)
```

在`__init__`方法中实例化所需的算子，并在`construct`中调用这些算子。这样，一个用于计算L1Loss的损失函数就定义好了。

代码中使用`nn.Cell`作为L1Loss的基类，最后在`construct`中调用基类提供的`predict, target`方法。`reduction`的合法参数有`mean`、`sum`和`none`，分别表示求均值、求和与输出原值。

给定一组预测值`predict`和真实值`target`，调用损失函数，就可以得到这组预测值和真实值之间的差异，如下所示：

```python
import numpy as np
from mindspore import Tensor

loss = L1Loss()
input_data = Tensor(np.array([0.1, 0.2, 0.3]).astype(np.float32))
target_data = Tensor(np.array([0.1, 0.2, 0.2]).astype(np.float32))

output = loss(input_data, target_data)
print(output)
```

以`Ascend`后端为例，输出结果如下：

```text
0.03333334
```

在定义损失函数时还可以继承损失函数的基类`Loss`。`Loss`提供了`get_loss`方法，用于对损失值求和或求均值，输出一个标量。

L1Loss使用`Loss`作为基类的定义如下：

In [1]:
import numpy as np
import mindspore.ops as ops
from mindspore.nn import LossBase

class L1Loss(LossBase):
    def __init__(self, reduction="mean"):
        super(L1Loss, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, base, target):
        x = self.abs(base - target)
        return self.get_loss(x)

首先，使用`Loss`作为L1Loss的基类，然后给`__init__`增加一个参数`reduction`，并通过`super`传给基类，最后在`construct`中调用基类提供的`get_loss`方法。`reduction`的合法参数有`mean`、`sum`和`none`，分别表示求均值、求和与输出原值。


## 损失函数与模型训练

接下来使用定义好的L1Loss进行模型训练。以简单的线性拟场景作为样例。

> 线性拟合详细介绍可参考教程[实现简单线性函数拟合](https://www.mindspore.cn/docs/programming_guide/zh-CN/r1.3/quick_start/linear_regression.html)。

### 定义数据集与网络

定义训练数据集生成函数，并增强为MindSpore可训练的数据类型。

- `get_data`：数据生成函数。
- `create_dataset`：将numpy数据转换为MindSpore可训练的函数，并构造数据集的batch增强方法。

In [2]:
from mindspore import dataset as ds

# 生成随机数
def get_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise = np.random.normal(0, 1)
        y = x * w + b + noise
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)

def create_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label'])
    dataset = dataset.batch(batch_size)
    return dataset

定义网络，`nn.Dense`将数据集定义为所有的函数。

In [3]:
import numpy as np
from mindspore.common.initializer import Normal
import mindspore.nn as nn

class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)

### 使用Model进行模型训练

`Model`是MindSpore提供的用于模型训练、评估和推理的高阶API。创建数据集并定义一个`Model`就可以使用`train`接口进行模型训练。接下来我们使用`Model`进行模型训练，并采用之前定义好的`L1Loss`作为此次训练的损失函数。

使用之前定义的`LinearNet`和`L1Loss`作为前向网络和损失函数，并选择MindSpore提供的`Momemtum`作为优化器。

In [4]:
net = LinearNet()
loss = L1Loss()
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

定义`Model`时需要指定前向网络、损失函数和优化器，`Model`内部会将它们关联起来，组成一张训练网。

In [5]:
from mindspore import Model

model = Model(net, loss, opt)

创建数据集，并调用`train`接口进行模型训练。

参数解释：

- `epoch`：训练数据集的迭代次数。
- `train_dataset`：训练数据集。
- `callbacks`：`model.train`的回调函数参数，也可不使用回调函数功能。
- `LossMonitor`：损失函数值监视器，用于打印训练过程中的模型损失值。
- `dataset_sink_mode`（bool）：数据下沉模式，可加快训练。Ascend和GPU平台支持开启该功能（True）。

In [6]:
from mindspore.train.callback import LossMonitor

ds_train = create_dataset(num_data=160)
model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor()], dataset_sink_mode=False)

epoch: 1 step: 1, loss is 8.042041
epoch: 1 step: 2, loss is 10.255897
epoch: 1 step: 3, loss is 11.757828
epoch: 1 step: 4, loss is 10.169484
epoch: 1 step: 5, loss is 8.056287
epoch: 1 step: 6, loss is 7.80862
epoch: 1 step: 7, loss is 8.432369
epoch: 1 step: 8, loss is 8.16351
epoch: 1 step: 9, loss is 6.4131327
epoch: 1 step: 10, loss is 4.8829217
